In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

from sklearn.cluster import KMeans 
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering

In [3]:
import sys

sys.path.append('../../src')



In [4]:
from base_exp_gen import generate_experiment as GEN_EXP
from clustering_algos import agglomerative_clustering, kmeans_clustering, box_clustering
from sklearn.model_selection import train_test_split as datasplit
from sklearn.ensemble import GradientBoostingRegressor as GDBR
from evaluations import computeATT_per_cluster, predict_cf, calculate_ite, get_homogeneity



In [5]:
np.random.seed(seed = 0)

In [6]:
pd.set_option("display.max_columns", None)


In [7]:
x_partition = [[i,i**2,2*i**4, 50000] for i in [2,4,5,10,12,20,25,33]]
def its(x):
    for i, (x1,x2,x3,x4) in enumerate(x[1:]):
        x[i+1][3]=(int(x[i][3]/(2*np.sqrt(x3/x[i][2]))))
    return x

In [8]:
a = its(x_partition)
exps = pd.DataFrame(a, columns = ['X-parts','Clusters', 'N', 'iters'])

In [9]:
name = 'kmeans'
exps.index.name = name

In [10]:
exps['mean-hom'] = 0
exps['std-hom'] = 0

In [11]:
exps

,X-parts,Clusters,N,iters,mean-hom,std-hom
kmeans,,,,,,
0,2,4,32,50000,0,0
1,4,16,512,6250,0,0
2,5,25,1250,2000,0,0
3,10,100,20000,250,0,0
4,12,144,41472,86,0,0
5,20,400,320000,15,0,0
6,25,625,781250,4,0,0
7,33,1089,2371842,1,0,0


In [12]:
import time

In [13]:
%%time
col_select = ['x0', 'x1']
gen = []
for i,row in exps.iterrows():
    c, n, iters = row.Clusters, row.N, row.iters
    homc = []
    print('running:',c,n)
    ss = time.time()
    for j in range(iters):
        ###gneerate data
        N,D = n,2
        kw_generate =  {'N':n, 'D':D,'f_gen_name': 'uniform_gen','low':0, 'high': 1}
        kw_cluster =  {'f_class_name':'circle_class', 'centers': [0.5, 0.5], 'r_small': 0.2, 'r_big':0.4}
        kw_treatment = {'f_treat_name' :'uniform_treat','choices':[0,1], 'probabilities':[0.5,0.5]}

        stats = np.array([[1, 0.75], [0, 0.75], [1, 0.75], [1, 0.75]])
        kw_outcome = {'f_outcome_name': 'outcome1','treatment':'Treatment', 'cls':'C', 'stats':stats}

        data = GEN_EXP(kw_generate, kw_cluster, kw_treatment, kw_outcome).dat
        data = calculate_ite(data.copy(), treatment = 'Treatment',
                     counterfactual = 'Ycf', outcome = 'Y', ite_name = 'ITE')
        
        ### cluster data
        data2,_ = kmeans_clustering(data, clusters=c,col_select = col_select, cluster_name = 'A')
        ### calc hom
        res = computeATT_per_cluster(data2.copy(), hom_name = 'HOM',weight_names = 'W',
                                     cluster_name  = "A", att = False)
        hom = (res['HOM']*res['W']).sum(axis = 0)
        homc.append(hom)
        
    ###get stats
    ee = time.time() - ss
    print('time:',ee)
    homcnp = np.array(homc)
    gen.append(homcnp)
    mean = np.mean(homcnp)
    std = np.std(homcnp)
    exps.loc[i,'mean-hom'] = mean
    exps.loc[i,'std-hom'] = std
    
    

running: 4 32
time: 3359.8274035453796
running: 16 512
time: 1103.7299070358276
running: 25 1250
time: 746.9820826053619
running: 100 20000
time: 23931.941823244095
running: 144 41472
time: 10043.633715391159
running: 400 320000
time: 5409.166598558426
running: 625 781250
time: 4231.756638765335
running: 1089 2371842
time: 4981.432255029678
CPU times: user 11d 19h 11min 15s, sys: 41min 10s, total: 11d 19h 52min 25s
Wall time: 14h 56min 48s


In [14]:
exps

,X-parts,Clusters,N,iters,mean-hom,std-hom
kmeans,,,,,,
0,2,4,32,50000,0.671420,0.060252
1,4,16,512,6250,0.744431,0.032001
2,5,25,1250,2000,0.804816,0.025722
3,10,100,20000,250,0.916408,0.007762
4,12,144,41472,86,0.921159,0.004070
5,20,400,320000,15,0.953636,0.001730
6,25,625,781250,4,0.964272,0.000340
7,33,1089,2371842,1,0.972441,0.000000


In [15]:
from pathlib import Path



In [16]:
p = './homog/'
Path('./homog/').mkdir(parents = True, exist_ok = True)
exps.to_csv(p+name+'.csv')

In [17]:
4*(3/4)**16+(1-4*(3/4)**16)*0.623

0.6381140744024888